In [13]:
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import os



In [21]:
final_data = pd.read_csv("final_data.csv")

In [18]:
final_data.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,Genre
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,hip hop
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,pop
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,reggae


In [ ]:
final_data = pd.DataFrame({})

def getJson(dir):
    with open(dir, 'r') as file:
        data = json.load(file)

    return data

def process_playlist(playlist, playlist_index, playlistCount):
    """Process a single playlist and return its tracks as a DataFrame."""
    local_tracks = []
    trackCount = len(playlist['tracks'])
    for f, track in enumerate(playlist['tracks'], start=1):
        local_tracks.append(track)
        # Note: printing from multiple threads may jumble output.
        #print(f"Progress: {f}/{trackCount} (on playlist {playlist_index}/{playlistCount})")
    return pd.DataFrame(local_tracks)

def getalltracks(data, max_workers=4):
    """Returns a DataFrame containing all tracks from all playlists using multithreading."""
    playlists = data['playlists']
    playlistCount = len(playlists)
    
    # Use ThreadPoolExecutor to process each playlist concurrently
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit a job for each playlist
        futures = [executor.submit(process_playlist, playlist, i, playlistCount)
                   for i, playlist in enumerate(playlists)]
        
        # Collect all the DataFrames as they complete
        df_list = []
        for future in as_completed(futures):
            try:
                df_list.append(future.result())
            except Exception as e:
                print("Error processing a playlist:", e)
    
    # Concatenate all DataFrames from the playlists into one
    retDict = pd.concat(df_list, axis=0, ignore_index=True)
    return retDict

files = os.listdir('Million Playlist')
tracks_list = []

i = 0

for file in files:
    tracks = getalltracks(getJson(f'Million Playlist\{file}'))

    tracks_list.append(tracks)

    i+=1

    print(f"Progress: {i}/{len(files)}")


final_data = pd.concat(tracks_list, axis=0, ignore_index=True)

In [ ]:
ACCESS_TOKEN = "BQDFavo8fE5NhzayKb-MCcj4WoWjTc6emYhjg0FOSCmaaT30uVNglYebrEmTF7tU6SpMLMgEYTvi9B9s1nUuU0PafajR7fgpLnPGqCMf9bb2KD_npAtc4FToG2Clxc9eQXPuoko9tVE"

def get_artist_genre(artist_id):
    """Fetches genres for a given artist ID from Spotify API."""
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        artist_data = response.json()
        return artist_id, artist_data.get("genres", [])  # Return (artist_id, genres)
    elif response.status_code == 429:  # Handle Rate Limiting
        print("Rate limited! Waiting before retrying...")
        time.sleep(5)  # Wait before retrying
        return get_artist_genre(artist_id)  # Retry request
    else:
        print(f"Error fetching artist {artist_id}: {response.status_code}")
        return artist_id, None  # Return None for failed requests
    

rows_with_none_artist_uri = final_data[final_data['Genre'].isna()]['artist_uri'].unique()
artist_ids = [artist.split(":")[2] for artist in rows_with_none_artist_uri]


    
i = 0
    
for artist in artist_ids:
    final_data.loc[final_data['artist_uri'] == artist, 'Genre'] = get_artist_genre(artist)
    i+=1

    print(f"progress: {i}/{len(artist_ids)}")
    

Rate limited! Waiting before retrying...
Rate limited! Waiting before retrying...
Rate limited! Waiting before retrying...
Rate limited! Waiting before retrying...


KeyboardInterrupt: 

In [ ]:
ACCESS_TOKEN = "BQDPMSG5-eJEgjp9MN1fXsWrIsVijDFvjmzhAcEVKicOPSJNjf30iWqx1ZfjLqtS5XtdZS0qDDTIEfnBXnkFWI0TmhIK9N1JL_zGjsJgIZ0DLt5_Zsm8yEvXO9ZNUF3Wz2k996pQCEw"
endpoint = "https://api.spotify.com/v1/albums/" # id 

def getReleaseYear(albumId):
    response = requests.get(endpoint + albumID, headers={"Authorization":f"Bearer {ACCESS_TOKEN}"})

    if response.status_code == 200:
        return response.json()['release_date'].split('-')[0]
    elif response.status_code == 429:
        retry = response.headers.get("Retry-After")
        print(f"Rate limited! Waiting {retry}")
        time.sleep(retry)
        return getReleaseYear(albumId)
    elif response.status_code == 404:
        print(f"Failed for {albumID}, content doesn't exist")
        return None
    

    print(response.json())
    response.json()['release_date'].split('-')[0]

albums = final_data[final_data['releaseYear'].isna()]['album_uri'].unique()

i = 0

for album in albums:
    albumID = album.split(":")[2]

    releaseYear = getReleaseYear(albumID)

    if releaseYear:
        final_data.loc[final_data['album_uri'] == album, "releaseYear"] = int(releaseYear)

    i+=1

    print(f"Progress: {i}/{len(albums)}")




Failed for 324ychEJu5FNpV2lI6hLxI, content doesn't exist
Progress: 1/22833
Progress: 2/22833
Progress: 3/22833
Progress: 4/22833
Progress: 5/22833
Progress: 6/22833
Progress: 7/22833
Progress: 8/22833
Progress: 9/22833
Progress: 10/22833
Progress: 11/22833
Progress: 12/22833
Progress: 13/22833
Progress: 14/22833
Progress: 15/22833
Progress: 16/22833
Progress: 17/22833
Progress: 18/22833
Progress: 19/22833
Progress: 20/22833
Progress: 21/22833
Progress: 22/22833
Progress: 23/22833
Progress: 24/22833
Progress: 25/22833
Progress: 26/22833
Progress: 27/22833
Progress: 28/22833
Progress: 29/22833
Progress: 30/22833
Progress: 31/22833
Progress: 32/22833
Progress: 33/22833
Progress: 34/22833
Progress: 35/22833
Progress: 36/22833
Progress: 37/22833
Progress: 38/22833
Progress: 39/22833
Progress: 40/22833
Progress: 41/22833
Progress: 42/22833
Progress: 43/22833
Progress: 44/22833
Progress: 45/22833
Progress: 46/22833
Progress: 47/22833
Progress: 48/22833
Progress: 49/22833
Progress: 50/22833
Pr

TypeError: 'str' object cannot be interpreted as an integer